# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 13**: Clustering with k-means

**Fecha**: 08/05/2025

**Equipo**:
- Luis Raúl Acosta Mendoza
- Samantha Abigail Quintero Valadez 
- Arturo Benjamin Vergara Romo

**Profesor**: Pablo Camarillo Ramirez

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Wine-K-means-Gatubelxs") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

25/05/08 20:19:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
from gatubelxs.spark_utils import SparkUtils

def generate_schema(columns):
    return [(col, "float") for col in columns]

In [15]:
columns = [
    "Alcohol","Malic_Acid","Ash","Ash_Alcanity","Magnesium","Total_Phenols","Flavanoids","Nonflavanoid_Phenols","Proanthocyanins","Color_Intensity","Hue","OD280","Proline"
]

schema = SparkUtils.generate_schema(generate_schema(columns))

wine_df = spark \
    .read \
    .schema(schema) \
    .option("header", "true") \
    .csv("/home/jovyan/notebooks/data/wine-clustering.csv")

wine_df.show(40)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|  14.23|      1.71|2.43|        15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92| 1065.0|
|   13.2|      1.78|2.14|        11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4| 1050.0|
|  13.16|      2.36|2.67|        18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17| 1185.0|
|  14.37|      1.95| 2.5|        16.8|    113.0|         3.85|      3.49|                0.24|           2.18|            7.

#### Assemble features into a single vector

In [16]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=columns, outputCol="features")
assembled_win_df = assembler.transform(wine_df)

#### K-Means

In [22]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

def eval_k_means(k):
    k_means = KMeans().setK(k).setSeed(19)
    model = k_means.fit(assembled_win_df)
    predictions = model.transform(assembled_win_df)

    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    print(f"Silhoutte score: {silhouette}")

In [23]:
k_values = [2,10,15,20]

for k in k_values:
    eval_k_means(k)

Silhoutte score: 0.821360351333219
Silhoutte score: 0.6099911274077665
Silhoutte score: 0.6663309347635443
Silhoutte score: 0.6341822740902026


In [7]:
sc.stop()